In [91]:
%pip install --upgrade --quiet  yandexcloud langchain-community yandex-cloud-ml-sdk


I0000 00:00:1758971185.747697   21116 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


Note: you may need to restart the kernel to use updated packages.


Заполняем словарь документов

In [92]:
documents = [
    {
        "id": 166213,
        "title": "Изменить формат общего шаблона excel при скачивании файлов",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/166213/"
    },

    {
        "id": 151543,
        "title": "Реализовать страницу со списком акселераторов",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/151543/"
    },

    {
        "id": 196677,
        "title": "[Bug][Major] Доработать статусы заявок в Акселераторах",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/196677/"
    },

    {
        "id": 181744,
        "title": "Реализация группы прав \"Менеджер Акселератора партнер\"",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/181744/"
    },

    {
        "id": 190897,
        "title": "Доработка кабинета Акселератора, добавление внешних модераторов",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/190897/"
    },

    {
        "id": 184464,
        "title": "Доработка ролевой модели в Акселерации (Демо)",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/184464/"
    },

    {
        "id": 187834,
        "title": "Добавление рассылки видеозаписи прошедшего мероприятия",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/187834/"
    },
]

обогащаем documents текстами задач

In [93]:
import os

# Добавляем поле context с текстом из .md файлов
for doc in documents:
    file_name = f"{doc['id']}.md"
    file_path = os.path.join("tasks", file_name)

    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            doc["context"] = f.read()
    else:
        doc["context"] = ""

    # Создаем поле search как объединение title и context
    doc["search"] = f"{doc['title']} {doc['context']}"



In [94]:
import pandas as pd # если нет импорта пандаса, то остальной код не имеет смысла

# создаем из наших документов датафрейм
df = pd.DataFrame(documents)
df.head()

,id,title,url,context,search
0,166213,Изменить формат общего шаблона excel при скачи...,https://inside.notamedia.ru/company/personal/u...,Требуется изменить формат excel общего шаблона...,Изменить формат общего шаблона excel при скачи...
1,151543,Реализовать страницу со списком акселераторов,https://inside.notamedia.ru/company/personal/u...,"1. Необходимо сделать активной кнопку ""Акселер...",Реализовать страницу со списком акселераторов ...
2,196677,[Bug][Major] Доработать статусы заявок в Аксел...,https://inside.notamedia.ru/company/personal/u...,Стенд: все площадки\n\n1) Кейс 1\n- Создать Ак...,[Bug][Major] Доработать статусы заявок в Аксел...
3,181744,"Реализация группы прав ""Менеджер Акселератора ...",https://inside.notamedia.ru/company/personal/u...,"Требуется реализовать новую группу прав ""Менед...","Реализация группы прав ""Менеджер Акселератора ..."
4,190897,"Доработка кабинета Акселератора, добавление вн...",https://inside.notamedia.ru/company/personal/u...,Требуется в кабинете Акселератора доработать д...,"Доработка кабинета Акселератора, добавление вн..."


In [95]:
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS


loader = DataFrameLoader(df, page_content_column='search')
documents = loader.load()

print(documents)

[Document(metadata={'id': 166213, 'title': 'Изменить формат общего шаблона excel при скачивании файлов', 'url': 'https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/166213/', 'context': 'Требуется изменить формат excel общего шаблона при скачивании файлов. Изменить с формата "Веб-страница" https://pastenow.ru/GMPC8 на xls.\n\nОтчеты в xls в блоке Бизнес-образования https://pastenow.ru/GMPBC\n\nОценка: 1 р.д.'}, page_content='Изменить формат общего шаблона excel при скачивании файлов Требуется изменить формат excel общего шаблона при скачивании файлов. Изменить с формата "Веб-страница" https://pastenow.ru/GMPC8 на xls.\n\nОтчеты в xls в блоке Бизнес-образования https://pastenow.ru/GMPBC\n\nОценка: 1 р.д.'), Document(metadata={'id': 151543, 'title': 'Реализовать страницу со списком акселераторов', 'url': 'https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/151543/', 'context': '1. Необходимо сделать активной кнопку "Акселерация" в разделе Бизнес-комп

In [96]:
# # создаем сплиттер документов, чтобы уложиться в лимит по токенам, в нашем случае это не очень полезный шаг

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

print(texts)

[Document(metadata={'id': 166213, 'title': 'Изменить формат общего шаблона excel при скачивании файлов', 'url': 'https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/166213/', 'context': 'Требуется изменить формат excel общего шаблона при скачивании файлов. Изменить с формата "Веб-страница" https://pastenow.ru/GMPC8 на xls.\n\nОтчеты в xls в блоке Бизнес-образования https://pastenow.ru/GMPBC\n\nОценка: 1 р.д.'}, page_content='Изменить формат общего шаблона excel при скачивании файлов Требуется изменить формат excel общего шаблона при скачивании файлов. Изменить с формата "Веб-страница" https://pastenow.ru/GMPC8 на xls.\n\nОтчеты в xls в блоке Бизнес-образования https://pastenow.ru/GMPBC\n\nОценка: 1 р.д.'), Document(metadata={'id': 151543, 'title': 'Реализовать страницу со списком акселераторов', 'url': 'https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/151543/', 'context': '1. Необходимо сделать активной кнопку "Акселерация" в разделе Бизнес-комп

In [97]:
import os

YC_FOLDER_ID = os.getenv("folder_id")
YC_API_KEY = os.getenv("ya_token")


In [98]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.yandex import YandexGPTEmbeddings


# Создаем модель для генерации эмбеддингов
yandex_embeddings = YandexGPTEmbeddings(model="doc", folder_id=YC_FOLDER_ID, iam_token=YC_API_KEY)


In [ ]:
Создаем индекс FAISS

In [108]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.yandex import YandexGPTEmbeddings

# Создаем модель для генерации эмбеддингов
yandex_embeddings = YandexGPTEmbeddings(model="doc", folder_id=YC_FOLDER_ID, iam_token=YC_API_KEY)

# Проверяем наличие локального индекса
index_path = "faiss_index"
if os.path.exists(index_path):
    print(f"Загрузка существующего индекса из {index_path}")
    faiss_index = FAISS.load_local(index_path, yandex_embeddings, allow_dangerous_deserialization=True)
else:
    print("Создание нового индекса")
    # Создание векторного хранилища FAISS
    faiss_index = FAISS.from_texts(
        texts=[doc.page_content for doc in texts],  # Тексты для индексирования
        embedding=yandex_embeddings,                # Экземпляр Embeddings
        metadatas=[{"id": doc.metadata.get("id", ""), "url": doc.metadata.get("url", ""), "title": doc.metadata.get("title", "")} for doc in texts]  # Метаданные
    )
    # Сохранение индекса локально
    faiss_index.save_local(index_path)



Загрузка существующего индекса из faiss_index


Проверяем индекс

In [109]:
faiss_index.similarity_search_with_score('Где назначаются права пользователя?')

[(Document(id='ced1bd6b-574f-4856-82e7-4bbfd3dc09f5', metadata={'id': 181744, 'url': 'https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/181744/', 'title': 'Реализация группы прав "Менеджер Акселератора партнер"'}, page_content='Ссылка на вкладку:  https://test.smbn.ru/bitrix/admin/busi...ackers.php\n\n\n4. Участники Акселерации (только назначенные Акселераторы).\n\nДоступные возможности:\n\n    Просмотр участников и их статусов.   \n\nСсылка на вкладку:  https://test.smbn.ru/bitrix/admin/busi...ipants.php\n\nОписание назначения прав:  \n\n1. Назначение прав пользователю производится в админ панели. В случае, если нет назначенных Акселераторов для пользователя с данной ролью,  все вкладки пустые;\n\n2. Администратору доступно прикрепление «Менеджер Акселератора партнер» в карточку Акселератора, требуется создать поле «Менеджер Акселератора партнер» (расположить после поля «Менеджер Акселератора внешний»). Добавить возможно нескольких внешних партнеров Акселератора к

Создаем модель

In [110]:
from langchain_community.llms import YandexGPT

llm = YandexGPT(
    folder_id=YC_FOLDER_ID,
    iam_token=YC_API_KEY,
    model="yandexgpt",
    temperature=0.3,
    max_tokens=500
)

Проверяем на запросе с использованием FAISS

In [111]:
query = 'Где назначаются права пользователя?'

In [112]:
from langchain.chains import RetrievalQA

# создаем цепочку
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=faiss_index.as_retriever()
)

qa_chain.invoke(query)

{'query': 'Где назначаются права пользователя?',
 'result': 'Права пользователя назначаются в админ-панели.'}

Подключаем промпт

In [120]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Создаем шаблон промпта с полями из метаданных
prompt_template = PromptTemplate(
    input_variables=["question", "context"],
    template="""Ответь на вопрос, используя следующие документы:

Контекст:
{context}

Вопрос:
{question}

Ответ:"""
)

# цепочка с кастомным промтом
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

# Получаем релевантные документы из FAISS
relevants = faiss_index.similarity_search(query, k=2)

# Формируем контекст и метаданные
context = "\n\n".join([doc.page_content for doc in relevants])

# Отправляем запрос в LLMChain
result = llm_chain.invoke({
    "question": query,
    "context": context
})

print("Ответ:")
print(result["text"])

Ответ:
В админ-панели.


https://us.cloud.langfuse.com

In [ ]:
# from langfuse import Langfuse
#
# langfuse = Langfuse(
#   secret_key="sk-lf-119c31ed-0120-42a6-9a02-35501a8aa210",
#   public_key="pk-lf-566814a4-7fc6-4da2-a1f6-9cb1335ea5cf",
#   host="https://us.cloud.langfuse.com"
# )